# Download MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005

Short Name/Collection Key: Greenland Quarterly Velocity

Collection ID: C2627036252-NSIDC_ECS

Sources:

https://nsidc.org/data/nsidc-0727/versions/5

https://search.earthdata.nasa.gov/search?q=NSIDC-0727%20V005

In [13]:
import toolbox.IcesheetCHANGES as changes
import toolbox.collection as collection
import toolbox.download as download

In [14]:
# Print collections in the database (collection.py)
collection.print_collections()

Available collections: 

MEaSUREs Annual Antarctic Ice Velocity Maps V001
Short name: MEaSUREs Antarctic Annual Velocity

MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short name: MEaSUREs Greenland Quarterly Velocity

MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short name: MEaSUREs Greenland Monthly Velocity

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height, Version 2
Short name: ATL14 Antarctic Elevation

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Short name: ATL15 Antarctic Elevation

ATLAS/ICESat-2 L3A Sea Ice Freeboard V006
Short name: ATL10 Sea Ice Freeboard



SET collection key and region name

In [15]:
collection_key = 'MEaSUREs Greenland Quarterly Velocity'
region_name = 'Greenland'   # 'Greenland' or 'Antarctica'
data_type = 'velocity'      # 'velocity' or 'elevation'

In [16]:
collection_id = collection.collection(collection_key)
print('Collection ID: ', collection_id)

Collection ID:  C2627036252-NSIDC_ECS


Next, define two directories on your local drive as follows:

| directory | purpose | 
|-----------|---------|
|`project_folder` | This is the path where output data from the changes module will be stored - the data to be used directly for analysis. | 
|`data_folder` | This is the path where ice velocity and elevation data, from their respective sources, will be stored. The data_folder option was created to facilitate data storage on external drives. |

In [17]:
project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
data_folder='/Volumes/Seagate/CHANGES/data_repository/tutorial'

Initialize the AntarcticCHANGES object - this object will contain all pertinent information to initialize the data grids in your region of interest.

In [6]:
GC = changes.GreenlandCHANGES(project_folder, data_folder) 

In [18]:
GC.print_attributes()
GC.collection_info(collection_key)

Metadata path:	 /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Metadata
Icesheet name:	 Greenland
Posting:	 50 km
EPSG:		 3413
Collection ID:  C2627036252-NSIDC_ECS
Collection long name:	 MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Collection short name:	 MEaSUREs Greenland Quarterly Velocity
Data type:		 Velocity
Download path:		 /Volumes/Seagate/CHANGES/data_repository/tutorial/Greenland/Velocity/MEaSUREs Greenland Quarterly Velocity/Data


Set verbose output to true or false

## Obtain list of available files for download

In [10]:
# this function is from the ICESat2 data page
def cmr_filter_nested_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    all_urls = []
    for e in search_results['feed']['entry']:
        granule_id = e['producer_granule_id']
        if verbose:
            print(granule_id)
        urls = []
        # TODO can generalize this with an if statement for use with non-nested data, ie. measures_antarctic_annual
        urls.append(granule_id)
        for link in e['links']:
            if 'href' not in link:
                # Exclude links with nothing to download
                continue    # continue jumps to next iteration in the loop
            if 'inherited' in link and link['inherited'] is True:
                # Why are we excluding these links?
                continue
            if 'rel' in link and 'data#' not in link['rel']:
                # Exclude links which are not classified by CMR as "data" or "metadata"
                continue
            if 'title' in link and 'opendap' in link['title'].lower():
                # Exclude OPeNDAP links--they are responsible for many duplicates
                # This is a hack; when the metadata is updated to properly identify
                # non-datapool links, we should be able to do this in a non-hack way
                continue
            urls.append(link['href'])
        all_urls.append(urls)

    return all_urls

In [21]:
# Build and call the CMR API URL
cmr_query_url = 'https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=' + GC.collection_id + '&page_size=2000'
print('CMR request URL: ', cmr_query_url)

urls = download.get_response(cmr_query_url)

CMR request URL:  https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=C2627036252-NSIDC_ECS&page_size=2000
Successfully obtained 640 URLs.


## Download the data

In [25]:
download.run_download_measures_monthly_files(GC, urls)

Processing 640 folders
6 files downloaded successfully.
All files downloaded successfully.


[]